In [1]:
import pandas as pd
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 1000)

In [2]:
df = pd.read_csv('raw_data.csv', parse_dates=['datadate'])

# create Adjusted Close feature (ajexq: "Adjustment Factor", prccq: "Close Price")
df['Adjusted Close'] = df['prccq'] / df['ajexq']

# create EBIT feature (txtq: "Total Income Taxes", xintq: "Interest and Related Expense")
df['EBIT'] = df['niq'] + df['txtq'] + df['xintq']

# create EBIT/EV ratio (cshoq: "Common Shares Outstanding", dlttq: "Total Long Term Debt")
df['EBIT/EV'] = df['EBIT'] / (df['prccq'] * df['cshoq'] + df['dlttq'] + df['dlcq'] - df['cheq'])

df_dict = {
    'tic': 'Ticker',
    'datadate': 'Date', 
    
    'revtq': 'Revenue',
    'cogsq': 'Cost of Goods Sold',
    'xsgaq': 'SG&A',
    #EBIT goes here
    'niq': 'Net Income',
    'cheq': 'Cash and Cash Equivalents',
    'rectq': 'Recievables',
    'invtq': 'Inventories',
    'acoq': 'Other Current Assets',
    'ppegtq': 'PP&E',
    'aoq': 'Other Assets',
    'dlcq': 'Debt in Current Liabilities',
    'apq': 'Accounts Payable',
    'txpq': 'Taxes Payable',
    'lcoq': 'Other Current Liabilities',
    'ltq': 'Total Liabilities',
    
    # Adjusted Close goes here
    'cshtrq': 'Volume'
    #EBIT/EV ratio goes here
           }

# note: for exchg 11-NYSE, 12-AMEX, 14-NASDAQ-NMS  
drop_cols = ['fyearq', 'fqtr', 'costat', 'indfmt', 'consol', 'popsrc', 'datafmt', 'curcdq', 'txtq', 'xintq', 
             'datacqtr', 'datafqtr', 'mkvaltq', 'exchg', 'cshoq', 'dlttq', 'conm']

order_cols = ['gvkey', 'Ticker', 'Date', 'Revenue', 'Cost of Goods Sold', 'SG&A', 'EBIT', 'Net Income',
              'Cash and Cash Equivalents', 'Recievables', 'Inventories', 'Other Current Assets', 'PP&E',
              'Other Assets', 'Debt in Current Liabilities', 'Accounts Payable', 'Taxes Payable',
              'Other Current Liabilities', 'Total Liabilities', 'Adjusted Close',  'Volume', 'EBIT/EV']

df = df.drop(columns=drop_cols)
df = df.rename(index=str, columns=df_dict)
df = df[order_cols]

pd.set_option('display.max_columns', 50)

In [3]:
# remove dates prior to 1990
df = df[(df['Date'].dt.year >= 1990)]
print(df.shape)

(368062, 22)


In [4]:
# remove gvkeys that have any NaN values
grouped = df.groupby('gvkey')

clean = []
semi_clean = []

for gvkey, gvkey_df in grouped:
    total_nan = gvkey_df.isnull().values.sum()
    if total_nan == 0:
        clean.append(gvkey)
    elif total_nan <= 6 and gvkey_df[['EBIT/EV', 'Volume']].isnull().values.any() == False:
        semi_clean.append(gvkey)
        
print(len(clean))
print(len(semi_clean))

48
52


In [5]:
df = df[df['gvkey'].isin(clean) | df['gvkey'].isin(semi_clean)]
df.fillna(method='ffill', inplace=True)
display(df)

/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3787: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)


,gvkey,Ticker,Date,Revenue,Cost of Goods Sold,SG&A,EBIT,Net Income,Cash and Cash Equivalents,Recievables,Inventories,Other Current Assets,PP&E,Other Assets,Debt in Current Liabilities,Accounts Payable,Taxes Payable,Other Current Liabilities,Total Liabilities,Adjusted Close,Volume,EBIT/EV
2845,1209,APD,1990-03-31,713.100,359.300,182.300,110.100,60.600,94.200,396.000,246.300,56.600,4785.700,423.000,75.500,310.500,1.200,145.900,2051.600,13.062500,1.017510e+07,0.028919
2846,1209,APD,1990-06-30,712.800,349.300,184.500,107.300,56.800,53.000,401.900,242.000,61.200,4966.500,534.500,137.900,316.800,12.300,156.100,2165.100,14.125000,1.308440e+07,0.025749
2847,1209,APD,1990-09-30,816.300,442.300,192.800,113.800,63.100,74.400,437.300,246.900,78.700,5010.200,577.100,103.400,350.800,16.900,151.900,2211.100,11.218750,8.326400e+06,0.032678
2848,1209,APD,1990-12-31,724.600,355.700,187.100,110.500,60.600,80.300,404.500,271.200,93.800,5191.100,603.700,120.600,365.300,31.800,151.100,2270.000,13.687500,8.338598e+06,0.027214
2849,1209,APD,1991-03-31,754.700,378.000,192.100,118.600,65.800,71.400,404.300,265.100,83.500,5139.400,609.500,156.700,349.800,15.400,179.400,2222.300,16.937500,1.003360e+07,0.024855
2850,1209,APD,1991-06-30,713.400,346.400,188.100,114.700,64.100,75.300,396.000,267.500,86.700,5137.000,622.600,107.400,326.100,21.100,173.700,2198.900,16.843750,6.357597e+06,0.024122
2851,1209,APD,1991-09-30,738.400,355.800,199.300,103.800,58.400,104.400,421.300,283.800,91.700,5332.200,703.700,190.400,389.200,11.100,193.800,2386.200,16.781250,8.011398e+06,0.021611
2852,1209,APD,1991-12-31,760.500,366.600,193.600,117.600,64.200,158.400,441.600,307.500,98.600,5502.700,726.600,287.200,341.300,18.400,190.300,2472.697,18.468750,6.773199e+06,0.022384
2853,1209,APD,1992-03-31,793.300,385.100,199.700,126.500,67.300,107.200,453.600,312.700,96.500,5471.600,704.100,219.100,349.900,14.500,188.400,2357.498,21.187500,1.713770e+07,0.021757
2854,1209,APD,1992-06-30,824.700,419.400,201.600,122.700,68.400,129.100,483.800,315.600,89.600,5682.000,733.900,163.900,385.300,10.000,207.300,2423.698,21.250000,1.473400e+07,0.021165


In [6]:
df.to_csv('100_clean.csv')